# Test code

In [1]:
# Community simulator package
from IPython.display import Image
from community_simulator import *
from community_simulator.usertools import *
from community_simulator.visualization import *
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends import backend_pdf as bpdf
import numpy as np
import scipy as sp
colors = sns.color_palette()
%matplotlib inline

# Community selection package
from community_selection import *
from community_selection.A_experiment_functions import *
from community_selection.B_community_phenotypes import *
from community_selection.C_selection_algorithms import *
from community_selection.D_migration_algorithms import *
from community_selection.usertools import *

In [2]:
# Make dynanmics by default we will use the microbial consumer resource model
def dNdt(N,R,params):
    return MakeConsumerDynamics(assumptions)(N,R,params)
def dRdt(N,R,params):
    return MakeResourceDynamics(assumptions)(N,R,params)
dynamics = [dNdt,dRdt]

# Global parameters
## Default parameters from community-simulator
## !!!Don't touch this dictionary!!!
assumptions = a_default.copy() # Start with default parameters

## Update parameters for community-selection
assumptions.update({
    'SA': 2100 * np.ones(1), #Number of species in each specialist family (here, 3 families of 60 species)
    'MA': 90 * np.ones(1), #Number of resources in each class
    'Sgen': 0, #Number of generalist species (unbiased sampling over alll resource classes)
    "n_wells": 140,
    "m": 0, # Mortality
    "scale": 10**6,  #scale is a conversion factor specifying the number of individual microbial cells present when N = 1.
    "sigma" : 1, # Standard deviation for drawing specifc speices/interaction function
    "alpha": 1, # Scaling factor between species- and interaction-specific function variances
    "l": 0, # Set leakage function to 0 to switch off cross-feeding
    "response": "type III",
    "sigma_max": 5,
    'R0_food': 1000, # Total amount of supplied food
    "rich_medium": True, # Number of food types passed to R0
    "binary_threshold": 1,  
    # The parameters below will be passed to params_simulation
    "n_propagation": 1, # Length of propagation, or hours within a growth cycle
    "n_transfer": 20, # Number of total transfer, or number of passage
    "n_transfer_selection": 10, # Number of transfer implementing seleciton regimes
    "dilution": 1/1000, # Dilution factor at every transfer
    "n_inoc": 10**6,  #Number of cells sampled from the regional species at start
    "selected_function": "f1_additive"
})


In [3]:
def sample_from_pool_richness(plate_N, scale = 10**6):
    S_tot = plate_N.shape[0] # Total number of species in the pool
    
    # Manipulated inocula sizes    
    inocula_size = list()
    for i in range(7):
        inocula_size = inocula_size + [2**i]*20
        
    consumer_index = plate_N.index # Consumer index
    well_names = ["W" + str(i) for i in range(len(inocula_size))] # Well index

    N0 = np.zeros((plate_N.shape[0], len(inocula_size))) # Make empty plate
    
    for k in range(len(inocula_size)):
        pool = np.random.power(0.01, size = S_tot) # Power-law distribution
        pool = pool/np.sum(pool) # Normalize the pool
        consumer_list = np.random.choice(S_tot, size = inocula_size[k], replace = True, p = pool) # Draw from the pool
        my_tab = pd.crosstab(index = consumer_list, columns = "count") # Calculate the cell count
        N0[my_tab.index.values,k] = np.ravel(my_tab.values / scale) # Scale to biomass

    # Make data.frame
    N0 = pd.DataFrame(N0, index = consumer_index, columns = well_names)

    return N0
#plate_N = sample_from_pool_richness(plate.N)

In [4]:
for seed_temp in range(1, 11):

    #seed_temp = 2
    params, params_simulation = prepare_experiment(assumptions, seed = seed_temp)

    data_directory = "data/test/"
    list_phenotypes = ["f5a_invader_growth"]
    list_algorithms = ["positive_control"]

    j = 0
    i = 0
    params_simulation.update({"selected_function": list_phenotypes[j]}) # selected function
    algorithms = make_algorithms(params_simulation)

    params_algorithm = algorithms[algorithms["algorithm_name"] == list_algorithms[i]]
    file_name = data_directory + "SP" + str(seed_temp) + "-" + list_algorithms[i]
    assembly_type = str(list_algorithms[i])
    write_composition = True
    write_function = True



    # Print out the algorithms
    print("\nAlgorithm: "+ params_algorithm["algorithm_name"][0])
    print("\n")
    print(params_algorithm[["transfer", "community_phenotype", "selection_algorithm", "migration_algorithm"]].to_string(index = False))

    # Set seeds
    np.random.seed(2)

    # Make initial state
    init_state = MakeInitialState(assumptions)

    # Make plate
    plate = Community(init_state, dynamics, params, scale = assumptions["scale"], parallel = True) 

    # Update the community composition by sampling from the pool
    print("\nGenerating initial plate")
    plate.N = sample_from_pool_richness(plate.N, scale = assumptions["scale"])

    # Update the supplied resource if assumptions["rich_medium"]
    if assumptions["rich_medium"]:
        plate.R = make_rich_medium(plate.R, assumptions)
        plate.R0 = make_rich_medium(plate.R, assumptions) # R0 for refreshing media on passaging if "refresh_resoruce" is turned on

    # Add the attributes that are essential to the function measurement to the plate objects 
    print("\nAdding attributes that are essential to the community function to the plate object")
    plate = add_community_function(plate, dynamics, assumptions, params, params_simulation, params_algorithm)

    # Empty list for saving data
    plate_data_list = list() # Plate composition
    community_function_list = list() # Community function

    # Save the inocula composition
    plate_data = reshape_plate_data(plate, transfer_loop_index = 0, assembly_type = assembly_type, community_function_name = params_algorithm["community_phenotype"][0]) # Initial state
    plate_data_list.append(plate_data)

    # Save the initial function
    community_function = globals()[params_algorithm["community_phenotype"][0]](plate, assumptions = assumptions) # Community phenotype
    richness = np.sum(plate.N >= 1/assumptions["scale"], axis = 0) # Richness
    biomass = list(np.sum(plate.N, axis = 0)) # Biomass
    function_data = reshape_function_data(community_function_name = params_algorithm["community_phenotype"][0], community_function = community_function, richness = richness, biomass = biomass, transfer_loop_index = 0, assembly_type = assembly_type)        
    community_function_list.append(function_data) # Transfer = 0 means that it's before selection regime works upon


    # Output the plate composition and community functions if write_composition set True
    if write_composition == True:
        plate_data.to_csv(file_name + "-" + params_algorithm["community_phenotype"][0] + "-T" + "{:02d}".format(0) + "-composition.txt", index = False)
    if write_function == True:
        function_data.to_csv(file_name + "-" + params_algorithm["community_phenotype"][0] + "-T" + "{:02d}".format(0) + "-function.txt", index = False)


    print("\nStart propogation")
    # Run simulation
    for i in range(0, params_simulation["n_transfer"]):
        # Algorithms used in this transfer
        phenotype_algorithm = params_algorithm["community_phenotype"][i]
        selection_algorithm = params_algorithm["selection_algorithm"][i]
        migration_algorithm = params_algorithm["migration_algorithm"][i]

        # Print the propagation progress
        print("Transfer " + str(i+1))

        # Propagation
        plate.Propagate(params_simulation["n_propagation"])

        # Append the composition to a list
        plate_data = reshape_plate_data(plate, transfer_loop_index = i + 1, assembly_type = assembly_type, community_function_name = phenotype_algorithm) # Transfer = 0 means that it's before selection regime works upon
        plate_data_list.append(plate_data)

        # Community phenotype, richness, and biomass
        community_function = globals()[phenotype_algorithm](plate, assumptions = assumptions) # Community phenotype
        richness = np.sum(plate.N >= 1/assumptions["scale"], axis = 0) # Richness
        biomass = list(np.sum(plate.N, axis = 0)) # Biomass
        function_data = reshape_function_data(community_function_name = phenotype_algorithm, community_function = community_function, richness = richness, biomass = biomass, transfer_loop_index = i + 1 , assembly_type = assembly_type)        
        community_function_list.append(function_data) # Transfer = 0 means that it's before selection regime works upon

        # Output the plate composition and community functions if write_composition set True
        if write_composition == True:
            plate_data.to_csv(file_name + "-" + phenotype_algorithm + "-T" + "{:02d}".format(i + 1) + "-composition.txt", index = False) # Transfer = 0 means that it's before selection regime works upon
        if write_function == True:
            function_data.to_csv(file_name + "-" + phenotype_algorithm + "-T" + "{:02d}".format(i + 1) + "-function.txt", index = False)

        # Passage and tranfer matrix
        transfer_matrix = globals()[selection_algorithm](community_function)
        plate.Passage(transfer_matrix * params_simulation["dilution"])

        # Migration
        m = globals()[migration_algorithm](community_function) 
        plate.N = migrate_from_pool(plate, migration_factor = m, scale = assumptions["scale"], inocula = params_simulation["n_inoc"])



Algorithm: positive_control


 transfer community_phenotype selection_algorithm migration_algorithm
        1  f5a_invader_growth        no_selection        no_migration
        2  f5a_invader_growth        no_selection        no_migration
        3  f5a_invader_growth        no_selection        no_migration
        4  f5a_invader_growth        no_selection        no_migration
        5  f5a_invader_growth        no_selection        no_migration
        6  f5a_invader_growth        no_selection        no_migration
        7  f5a_invader_growth        no_selection        no_migration
        8  f5a_invader_growth        no_selection        no_migration
        9  f5a_invader_growth        no_selection        no_migration
       10  f5a_invader_growth        no_selection        no_migration
       11  f5a_invader_growth        no_selection        no_migration
       12  f5a_invader_growth        no_selection        no_migration
       13  f5a_invader_growth        no_selection        no


Adding attributes that are essential to the community function to the plate object
Sampling invader (resident) community

Stabilizing the invader (resident) community. Passage for 10 transfers. The plate has  2100  wells
Passaging invader (resident) community. Transfer 1
Passaging invader (resident) community. Transfer 2
Passaging invader (resident) community. Transfer 3
Passaging invader (resident) community. Transfer 4
invader index = [1058]
Finished passaging the invader (resident) community
The community has 1 species
The community has initial biomass 1e-06 and reaches total biomass 262.1868266745989
The invader species (or dominant species in the resident community) has the biomass  262.1868266745989  at equilibrium

Start propogation
Transfer 1
Transfer 2
Transfer 3
Transfer 4
Transfer 5
Transfer 6
Transfer 7
Transfer 8
Transfer 9
Transfer 10
Transfer 11
Transfer 12
Transfer 13
Transfer 14
Transfer 15
Transfer 16
Transfer 17
Transfer 18
Transfer 19
Transfer 20

Algorithm: positi


Start propogation
Transfer 1
Transfer 2
Transfer 3
Transfer 4
Transfer 5
Transfer 6
Transfer 7
Transfer 8
Transfer 9
Transfer 10
Transfer 11
Transfer 12
Transfer 13
Transfer 14
Transfer 15
Transfer 16
Transfer 17
Transfer 18
Transfer 19
Transfer 20

Algorithm: positive_control


 transfer community_phenotype selection_algorithm migration_algorithm
        1  f5a_invader_growth        no_selection        no_migration
        2  f5a_invader_growth        no_selection        no_migration
        3  f5a_invader_growth        no_selection        no_migration
        4  f5a_invader_growth        no_selection        no_migration
        5  f5a_invader_growth        no_selection        no_migration
        6  f5a_invader_growth        no_selection        no_migration
        7  f5a_invader_growth        no_selection        no_migration
        8  f5a_invader_growth        no_selection        no_migration
        9  f5a_invader_growth        no_selection        no_migration
       10  f5a_inva

In [5]:
for seed_temp in range(1, 11):

    #seed_temp = 2
    params, params_simulation = prepare_experiment(assumptions, seed = seed_temp)

    data_directory = "data/test/"
    list_phenotypes = ["f5a_invader_growth"]
    list_algorithms = ["simple_screening"]

    j = 0
    i = 0
    params_simulation.update({"selected_function": list_phenotypes[j]}) # selected function
    algorithms = make_algorithms(params_simulation)

    params_algorithm = algorithms[algorithms["algorithm_name"] == list_algorithms[i]]
    file_name = data_directory + "SP" + str(seed_temp) + "-" + list_algorithms[i]
    assembly_type = str(list_algorithms[i])
    write_composition = True
    write_function = True



    # Print out the algorithms
    print("\nAlgorithm: "+ params_algorithm["algorithm_name"][0])
    print("\n")
    print(params_algorithm[["transfer", "community_phenotype", "selection_algorithm", "migration_algorithm"]].to_string(index = False))

    # Set seeds
    np.random.seed(2)

    # Make initial state
    init_state = MakeInitialState(assumptions)

    # Make plate
    plate = Community(init_state, dynamics, params, scale = assumptions["scale"], parallel = True) 

    # Update the community composition by sampling from the pool
    print("\nGenerating initial plate")
    plate.N = sample_from_pool(plate.N, scale = assumptions["scale"], inocula = params_simulation["n_inoc"])
#    plate.N = sample_from_pool_richness(plate.N, scale = assumptions["scale"])

    # Update the supplied resource if assumptions["rich_medium"]
    if assumptions["rich_medium"]:
        plate.R = make_rich_medium(plate.R, assumptions)
        plate.R0 = make_rich_medium(plate.R, assumptions) # R0 for refreshing media on passaging if "refresh_resoruce" is turned on

    # Add the attributes that are essential to the function measurement to the plate objects 
    print("\nAdding attributes that are essential to the community function to the plate object")
    plate = add_community_function(plate, dynamics, assumptions, params, params_simulation, params_algorithm)

    # Empty list for saving data
    plate_data_list = list() # Plate composition
    community_function_list = list() # Community function

    # Save the inocula composition
    plate_data = reshape_plate_data(plate, transfer_loop_index = 0, assembly_type = assembly_type, community_function_name = params_algorithm["community_phenotype"][0]) # Initial state
    plate_data_list.append(plate_data)

    # Save the initial function
    community_function = globals()[params_algorithm["community_phenotype"][0]](plate, assumptions = assumptions) # Community phenotype
    richness = np.sum(plate.N >= 1/assumptions["scale"], axis = 0) # Richness
    biomass = list(np.sum(plate.N, axis = 0)) # Biomass
    function_data = reshape_function_data(community_function_name = params_algorithm["community_phenotype"][0], community_function = community_function, richness = richness, biomass = biomass, transfer_loop_index = 0, assembly_type = assembly_type)        
    community_function_list.append(function_data) # Transfer = 0 means that it's before selection regime works upon


    # Output the plate composition and community functions if write_composition set True
    if write_composition == True:
        plate_data.to_csv(file_name + "-" + params_algorithm["community_phenotype"][0] + "-T" + "{:02d}".format(0) + "-composition.txt", index = False)
    if write_function == True:
        function_data.to_csv(file_name + "-" + params_algorithm["community_phenotype"][0] + "-T" + "{:02d}".format(0) + "-function.txt", index = False)


    print("\nStart propogation")
    # Run simulation
    for i in range(0, params_simulation["n_transfer"]):
        # Algorithms used in this transfer
        phenotype_algorithm = params_algorithm["community_phenotype"][i]
        selection_algorithm = params_algorithm["selection_algorithm"][i]
        migration_algorithm = params_algorithm["migration_algorithm"][i]

        # Print the propagation progress
        print("Transfer " + str(i+1))

        # Propagation
        plate.Propagate(params_simulation["n_propagation"])

        # Append the composition to a list
        plate_data = reshape_plate_data(plate, transfer_loop_index = i + 1, assembly_type = assembly_type, community_function_name = phenotype_algorithm) # Transfer = 0 means that it's before selection regime works upon
        plate_data_list.append(plate_data)

        # Community phenotype, richness, and biomass
        community_function = globals()[phenotype_algorithm](plate, assumptions = assumptions) # Community phenotype
        richness = np.sum(plate.N >= 1/assumptions["scale"], axis = 0) # Richness
        biomass = list(np.sum(plate.N, axis = 0)) # Biomass
        function_data = reshape_function_data(community_function_name = phenotype_algorithm, community_function = community_function, richness = richness, biomass = biomass, transfer_loop_index = i + 1 , assembly_type = assembly_type)        
        community_function_list.append(function_data) # Transfer = 0 means that it's before selection regime works upon

        # Output the plate composition and community functions if write_composition set True
        if write_composition == True:
            plate_data.to_csv(file_name + "-" + phenotype_algorithm + "-T" + "{:02d}".format(i + 1) + "-composition.txt", index = False) # Transfer = 0 means that it's before selection regime works upon
        if write_function == True:
            function_data.to_csv(file_name + "-" + phenotype_algorithm + "-T" + "{:02d}".format(i + 1) + "-function.txt", index = False)

        # Passage and tranfer matrix
        transfer_matrix = globals()[selection_algorithm](community_function)
        plate.Passage(transfer_matrix * params_simulation["dilution"])

        # Migration
        m = globals()[migration_algorithm](community_function) 
        plate.N = migrate_from_pool(plate, migration_factor = m, scale = assumptions["scale"], inocula = params_simulation["n_inoc"])



Algorithm: simple_screening


 transfer  community_phenotype selection_algorithm migration_algorithm
        1  f6a_resident_growth        no_selection        no_migration
        2  f6a_resident_growth        no_selection        no_migration
        3  f6a_resident_growth        no_selection        no_migration
        4  f6a_resident_growth        no_selection        no_migration
        5  f6a_resident_growth        no_selection        no_migration
        6  f6a_resident_growth        no_selection        no_migration
        7  f6a_resident_growth        no_selection        no_migration
        8  f6a_resident_growth        no_selection        no_migration
        9  f6a_resident_growth        no_selection        no_migration
       10  f6a_resident_growth        no_selection        no_migration
       11  f6a_resident_growth        no_selection        no_migration
       12  f6a_resident_growth        no_selection        no_migration
       13  f6a_resident_growth        no_selec

Passaging invader (resident) community. Transfer 9
Passaging invader (resident) community. Transfer 10
invader index = [742]
Finished passaging the invader (resident) community
The community has 40 species
The community has initial biomass 1.0 and reaches total biomass 1000.1839614747053
The invader species (or dominant species in the resident community) has the biomass  170.39558489042466  at equilibrium
[742]

Start propogation
Transfer 1
[742]
Transfer 2
[742]
Transfer 3
[742]
Transfer 4
[742]
Transfer 5
[742]
Transfer 6
[742]
Transfer 7
[742]
Transfer 8
[742]
Transfer 9
[742]
Transfer 10
[742]
Transfer 11
[742]
Transfer 12
[742]
Transfer 13
[742]
Transfer 14
[742]
Transfer 15
[742]
Transfer 16
[742]
Transfer 17
[742]
Transfer 18
[742]
Transfer 19
[742]
Transfer 20
[742]

Algorithm: simple_screening


 transfer  community_phenotype selection_algorithm migration_algorithm
        1  f6a_resident_growth        no_selection        no_migration
        2  f6a_resident_growth        no_s


Adding attributes that are essential to the community function to the plate object
(2100, 140)
Sampling invader (resident) community

Stabilizing the invader (resident) community. Passage for 10 transfers. The plate has  96  wells
Passaging invader (resident) community. Transfer 1
Passaging invader (resident) community. Transfer 2
Passaging invader (resident) community. Transfer 3
Passaging invader (resident) community. Transfer 4
Passaging invader (resident) community. Transfer 5
Passaging invader (resident) community. Transfer 6
Passaging invader (resident) community. Transfer 7
Passaging invader (resident) community. Transfer 8
Passaging invader (resident) community. Transfer 9
Passaging invader (resident) community. Transfer 10
invader index = [436]
Finished passaging the invader (resident) community
The community has 43 species
The community has initial biomass 0.9999999999999999 and reaches total biomass 1000.1722304385579
The invader species (or dominant species in the resident


Adding attributes that are essential to the community function to the plate object
(2100, 140)
Sampling invader (resident) community

Stabilizing the invader (resident) community. Passage for 10 transfers. The plate has  96  wells
Passaging invader (resident) community. Transfer 1
Passaging invader (resident) community. Transfer 2
Passaging invader (resident) community. Transfer 3
Passaging invader (resident) community. Transfer 4
Passaging invader (resident) community. Transfer 5
Passaging invader (resident) community. Transfer 6
Passaging invader (resident) community. Transfer 7
Passaging invader (resident) community. Transfer 8
Passaging invader (resident) community. Transfer 9
Passaging invader (resident) community. Transfer 10
invader index = [371]
Finished passaging the invader (resident) community
The community has 33 species
The community has initial biomass 1.0 and reaches total biomass 1000.2860787102248
The invader species (or dominant species in the resident community) has

In [6]:
seed_temp = 1
params, params_simulation = prepare_experiment(assumptions, seed = seed_temp)

data_directory = "data/test/"
list_phenotypes = ["f6a_resident_growth"]
list_algorithms = ["positive_control"]

j = 0
i = 0
params_simulation.update({"selected_function": list_phenotypes[j]}) # selected function
algorithms = make_algorithms(params_simulation)

params_algorithm = algorithms[algorithms["algorithm_name"] == list_algorithms[i]]
file_name = data_directory + "SP" + str(seed_temp) + "-" + list_algorithms[i]
assembly_type = str(list_algorithms[i])
write_composition = True
write_function = True



# Print out the algorithms
print("\nAlgorithm: "+ params_algorithm["algorithm_name"][0])
print("\n")
print(params_algorithm[["transfer", "community_phenotype", "selection_algorithm", "migration_algorithm"]].to_string(index = False))

# Set seeds
np.random.seed(2)

# Make initial state
init_state = MakeInitialState(assumptions)

# Make plate
plate = Community(init_state, dynamics, params, scale = assumptions["scale"], parallel = True) 

# Update the community composition by sampling from the pool
print("\nGenerating initial plate")
plate.N = sample_from_pool_richness(plate.N, scale = assumptions["scale"])

# Update the supplied resource if assumptions["rich_medium"]
if assumptions["rich_medium"]:
    plate.R = make_rich_medium(plate.R, assumptions)
    plate.R0 = make_rich_medium(plate.R, assumptions) # R0 for refreshing media on passaging if "refresh_resoruce" is turned on

# Add the attributes that are essential to the function measurement to the plate objects 
print("\nAdding attributes that are essential to the community function to the plate object")
plate = add_community_function(plate, dynamics, assumptions, params, params_simulation, params_algorithm)





Algorithm: simple_screening


 transfer  community_phenotype selection_algorithm migration_algorithm
        1  f6a_resident_growth        no_selection        no_migration
        2  f6a_resident_growth        no_selection        no_migration
        3  f6a_resident_growth        no_selection        no_migration
        4  f6a_resident_growth        no_selection        no_migration
        5  f6a_resident_growth        no_selection        no_migration
        6  f6a_resident_growth        no_selection        no_migration
        7  f6a_resident_growth        no_selection        no_migration
        8  f6a_resident_growth        no_selection        no_migration
        9  f6a_resident_growth        no_selection        no_migration
       10  f6a_resident_growth        no_selection        no_migration
       11  f6a_resident_growth        no_selection        no_migration
       12  f6a_resident_growth        no_selection        no_migration
       13  f6a_resident_growth        no_selec

Process ForkPoolWorker-10:
Process ForkPoolWorker-15:
Process ForkPoolWorker-14:
Process ForkPoolWorker-12:
Process ForkPoolWorker-13:
Process ForkPoolWorker-11:
Process ForkPoolWorker-16:
Process ForkPoolWorker-9:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/cychang/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/cychang/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/cychang/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/cychang/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/cychang/anaconda3/lib/python3.

KeyboardInterrupt: 

  File "/Users/cychang/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/cychang/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/cychang/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/cychang/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/cychang/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/cychang/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/cychang/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/cychang/anaconda3/lib/python3.7/multiproce

  File "<ipython-input-2-b7175e79501e>", line 5, in dRdt
    return MakeResourceDynamics(assumptions)(N,R,params)
  File "<ipython-input-2-b7175e79501e>", line 5, in dRdt
    return MakeResourceDynamics(assumptions)(N,R,params)
  File "/Users/cychang/Desktop/Lab/community-simulator/community_simulator/usertools.py", line 346, in <lambda>
    +(J_out(R,params)/params['w']).T.dot(N))
KeyboardInterrupt
  File "/Users/cychang/Desktop/Lab/community-simulator/community_simulator/usertools.py", line 380, in <lambda>
    return lambda N,R,params: params['g']*N*(np.sum(J_growth(R,params),axis=1)-params['m'])
  File "/Users/cychang/Desktop/Lab/community-simulator/community_simulator/usertools.py", line 346, in <lambda>
    +(J_out(R,params)/params['w']).T.dot(N))
KeyboardInterrupt
  File "/Users/cychang/Desktop/Lab/community-simulator/community_simulator/usertools.py", line 346, in <lambda>
    +(J_out(R,params)/params['w']).T.dot(N))
  File "/Users/cychang/Desktop/Lab/community-simulator/commun